### Evaluating Retrieval-Augmented Generation (RAG) Applications: Importance and Impact  

Evaluating RAG-based applications is crucial to ensure the reliability, relevance, and efficiency of their responses. Unlike using LLMs by themselves, RAG-based applications combine the power of Large-Language-Models with vector stores and retrieval components, which introduces complexity. Hence, evaluation of different components in RAG pipelines is crucial. 

#### **Key Pitfalls in RAG Applications**  
- **Hallucination**: Even with retrieved context, LLMs can still generate inaccurate or misleading responses.  
- **Irrelevant Retrieval**: Poor retrieval results in the model relying on incomplete or incorrect context.  
- **Latency & Efficiency**: Combining retrieval and generation can introduce delays, requiring performance optimization. 

#### **Advantages of Proper Evaluation**  
- **Improved Accuracy**: Ensures the factual correctness of responses by using a knowledge base.  
- **Enhanced Relevance**: Helps refine retrieval mechanisms, ensuring responses are more context-aware.  
- **Robustness & Adaptability**: Identifies failure points, enabling continuous improvements to model behavior.  

#### **Impact on Future Development**  
A structured evaluation process enables better benchmarking, allowing for iterative improvements in retrieval models, embeddings, and LLM fine-tuning. It also promotes transparency in AI decision-making, paving the way for more explainable and ethical AI systems and thus gaining a wider adoption. 

In [ ]:
from dotenv import load_dotenv

"""
(OPTIONAL STEP)

If one wants to have a structured way of displaying the results of the evaluation, 
one can do so from here: https://app.ragas.io/. 
There one can sign-up and receive an app token. Once retrieved you can create a `env` file with the following content:
RAGAS_APP_TOKEN="apt.......-9f6ed"

Additionally, Langsmith can be used to trace the evaluation in real-time, since RAGAs is built with Langchain.
For that follow the link and follow the steps:
https://docs.ragas.io/en/latest/howtos/integrations/langsmith/#tracing-ragas-metrics
"""
load_dotenv()

### Loading the evaluation dataset

According to **RAGAs** an **evaluation dataset** is a homogeneous collection of **data samples** with the sole purpose of measuring the capabilities and performance of an AI application.

- **Structure**:

    - Contains either **SingleTurnSample** or **MultiTurnSample** objects instances, each of them representing a unique interaction between a **Scenario**.

    - **NOTE**: The dataset can contain **ONLY** a single type of sample type. They cannot be mixed together into a single dataset.

**Samples** represent a single unit of interaction with the underlying system. As mentioned they can be either **SingleTurnSample** or **MultiTurnSample**.

For this project I focus solely on **SingleTurnSample** objects, since I'm evaluating independent queries, not multi-turn interactions.

In [2]:
from ragas import EvaluationDataset

eval_dataset = EvaluationDataset.from_jsonl("dataset.jsonl")

In [3]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings

from ragas.evaluation import evaluate
from ragas.run_config import RunConfig
from ragas.cache import DiskCacheBackend
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

ollama_llm = OllamaLLM(
    model="llama3.1",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_ctx=24000,
    format="json"
)

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
    base_url="http://localhost:11434"
)

run_config = RunConfig(
    timeout = 43200, # twelve hourses, depending on GPU, model, testsize, etc -> can experinment
    max_wait = 30,
    log_tenacity = True
)

cacher = DiskCacheBackend(".cache")

llm = LangchainLLMWrapper(
    langchain_llm=ollama_llm,
    run_config=run_config,
    cache=cacher
)

embeddings = LangchainEmbeddingsWrapper(
    embeddings=ollama_embeddings,
    run_config=run_config,
    cache=cacher
)

### Metrics

Metrics in **RAGAs** can be classified depending on the mechanism they evaluate the application as:

- **LLM-based** - where a LLM is used to perform the evaluation. There might be more than one queries submitted to the LLM to evaluate the application on a single metric. Furthermore, this type of metrics mimic a user, since they tend to be quite non-deterministic and unpredictable. 

- **Non-LLM based** - where no LLM is required or used to perform the evaluation. These metrics tend to be much faster and predictable than the previous type. However, in my opinion are maybe not the best for evaluating a RAG system where a single query can be submitted multiple times and each time a new response might be generated.

The previously mentioned metric types can be further classified into **SingleTurnMetric** and **MultiTurnMetric** respectively. I would like to note again that in this project the **SingleTurnMetric** would be relevant and used.

---

### **Context Precision**

**Context Precision** is a crucial metric for evaluating the **retrieval capabilities** of a RAG (Retrieval-Augmented Generation) system. Since the **LLM's response** depends heavily on the **context retrieved from the knowledge base**, it is essential to have a reliable retrieval mechanism that fetches **only relevant** information.  

By achieving **high context precision**, the system ensures that the retrieved information is **more relevant**, leading to **more accurate responses** and **reduced hallucinations**.

> **TL;DR:** What fraction of the retrieved chunks are **actually relevant**?

---

### **Formula:**
For each retrieved chunk at **rank \( k \)**:

$\text{Precision@k} = \frac{\text{Number of relevant chunks at rank k}}{\text{Total number of retrieved chunks at rank k}}$

We then compute **Context Precision @ K** as the **mean of all Precision@k values**, weighted by relevance:

$\text{Context Precision @ K} = \frac{\sum_{k=1}^{K} \text{Precision@k} \times \text{Relevance}(k)}{\sum_{k=1}^{K} \text{Relevance}(k)}$

Where:
- **Precision@k** is the proportion of relevant chunks at rank \( k \).
- **Relevance(k)** is a binary indicator (1 if the chunk at rank \( k \) is relevant, 0 otherwise).
- **K** is the total number of retrieved chunks.

---

### **Example Calculation**
Suppose our **retriever fetches 4 chunks**, and **2 of them** are relevant. We calculate **Precision@k** at each rank:

| Rank \( k \) | Retrieved Chunk | Relevant? | Precision@k |
|-------------|----------------|------------|-------------|
| 1           | ✅ Relevant      | ✅ (1)      | \( $\frac{1}{1}$ = 1.00 \) |
| 2           | ❌ Not Relevant  | ❌ (0)      | \( $\frac{1}{2}$ = 0.50 \) |
| 3           | ✅ Relevant      | ✅ (1)      | \( $\frac{2}{3}$ = 0.67 \) |
| 4           | ❌ Not Relevant  | ❌ (0)      | \( $\frac{2}{4}$ = 0.50 \) |

Now, using the **Context Precision formula**:


$\text{Context Precision @ 4} = \frac{(1.00 \times 1) + (0.50 \times 0) + (0.67 \times 1) + (0.50 \times 0)}{2} = $

= $\frac{1.00 + 0 + 0.67 + 0}{2} = \frac{1.67}{2}$ = 0.835

Thus, **Context Precision @ 4 = 0.835**.

---

### **Why is This Useful?**

- **Evaluates Retriever Quality** → Ensures retrieved chunks contain **relevant** information.

- **Improves RAG Performance** → Helps **reduce hallucinations** and improves **LLM accuracy**.

In [ ]:
from ragas.metrics import LLMContextPrecisionWithoutReference

context_precision = LLMContextPrecisionWithoutReference()

### **Context Recall**

**Context Recall** measures how many of the **relevant documents (or pieces of information)** were successfully retrieved. This metric ensures that the retrieval process does not **miss important information** that could be used to generate a response. A **higher recall** means that fewer relevant documents were left out, making it crucial for applications that prioritize completeness over precision. Since recall focuses on **not missing relevant data**, it always requires a reference set for comparison.

### TL;DR:
> **How many of the relevant chunks were actually retrieved?**

---

### LLM-Based Context Recall

This metric evaluates recall by analyzing the **claims** in the reference (expected) response and checking whether they can be attributed to the retrieved context. In an ideal scenario, **all claims** in the reference answer should be **supported** by the retrieved context.

### **Formula**:

**Context Recall @ k** = \( $\frac{\text{Number of claims in the reference supported by the retrieved context}}{\text{Total number of claims in the reference}}$ \), where `k` is the rank

A recall score **closer to 1** indicates that most relevant data has been successfully retrieved, while a **lower recall** means that key information was missed.

In [ ]:
from ragas.metrics import LLMContextRecall

llm_context_recall = LLMContextRecall()

### **Context Entities Recall**

### TL;DR:
> **It measures the fraction of entities present in both context and reference relative to the total number of entities in the reference.**

---

### Definition:
`ContextEntityRecall` metric measures the recall of the retrieved context, based on the number of entities present in both `reference` and `retrieved_contexts`, relative to the total number of entities in the `reference`. In simple terms, it evaluates how well the retrieved contexts capture the entities from the original reference.

---

### Formula:

To compute this metric, we define two sets:

- **RE**: The set of entities in the reference.
- **RCE**: The set of entities in the retrieved contexts.

We determine the number of entities common to both sets (**RCE** $\cap$  **RE**) and divide it by the total number of entities in the reference (**RE**). The formula is:

$ \text{Context Entity Recall} = \frac{\text{Number of common entities between RCE and RE}}{\text{Total number of entities in RE}}$

In [ ]:
from ragas.metrics import ContextEntityRecall

context_entity_recall = ContextEntityRecall()

### **Noise Sensitivity**

### TL;DR:
> **How prone is the system to generating incorrect claims from retrieved contexts?**

---

### **Definition**:
**Noise Sensitivity** measures how often a system makes errors by providing incorrect responses when utilizing either relevant or irrelevant retrieved documents. This metric evaluates the robustness of a Retrieval-Augmented Generation (RAG) system against potentially misleading or noisy information.

---

### **Conceptual Insight**:
The metric fundamentally tests how easily an LLM can be "tricked" into generating factually incorrect responses. It distinguishes between two critical scenarios:

1. **Relevant Context Noise**: 
   - More subtle and dangerous
   - Noise is camouflaged within seemingly pertinent information
   - High risk of inadvertently incorporating incorrect claims

2. **Irrelevant Context Noise**:
   - A robust LLM should completely resist this
   - Contexts unrelated to the user's query
   - Zero tolerance for incorporating unrelated information

---

### LLM-Based Noise Sensitivity

The metric assesses noise sensitivity by decomposing the response and reference into individual claims, then analyzing:
- Whether claims are correct according to the ground truth
- Whether incorrect claims can be attributed to retrieved contexts (either relevant or irrelevant)

---

### **Calculation Approach**:

1. **Decompose Statements**: Break down the reference and response into individual claims
2. **Context Verification**: Check if claims can be supported by retrieved contexts
3. **Incorrect Claim Identification**: Determine which claims are unsupported by the ground truth

---

### **Formula**:

**Noise Sensitivity** = \( $\frac{\text{Number of incorrect claims in response}}{\text{Total number of claims}}$ \)

A score **closer to 0** indicates better performance, suggesting:
- Fewer incorrect claims
- Less influence from noisy or irrelevant contexts
- More robust response generation

---

### **Modes**:
- **Relevant Mode** (default): Focuses on noise in relevant contexts
- **Irrelevant Mode**: Analyzes noise from irrelevant retrieved documents

---

### **Evaluation Expectations**:
- **Relevant Context**: Some noise tolerance, but should be minimal
- **Irrelevant Context**: Virtually zero noise should be incorporated
  - A high-quality LLM should completely disregard irrelevant information
  - Any noise from irrelevant contexts indicates a significant vulnerability

---

### **Example Scenario**:
Consider a query about the Life Insurance Corporation of India (LIC):
- **Reference**: Describes LIC as the largest insurance company established in 1956
- **Response**: Includes an unsubstantiated claim about LIC's financial stability
- **Retrieved Contexts**: Mix of relevant and irrelevant information
- **Noise Sensitivity**: Measures how the response incorporates unverified claims

### **Key Insights**:
- Helps identify system vulnerabilities to hallucination
- Provides a quantitative measure of response reliability
- Distinguishes between subtle (relevant) and gross (irrelevant) information distortions

In [ ]:
from ragas.metrics import NoiseSensitivity

noise_sensitivity = NoiseSensitivity()

### **Response Relevancy**

### TL;DR:
> **How well does the answer address the original user query?**

---

### **Definition**:
**Response Relevancy** measures how closely an AI-generated response aligns with the original user input. It evaluates the answer's ability to directly and appropriately address the user's question, penalizing responses that are incomplete, off-topic, or include unnecessary details. It doesn't judge the **factual accuracy**.

---

### **Conceptual Insight**:
The metric focuses on the semantic alignment between:
- The original user query
- The generated response

Key evaluation criteria:
1. **Direct Address**: Does the answer directly tackle the user's question?
2. **Information Completeness**: Does the response provide sufficient information?
3. **Topic Coherence**: Does the answer stay focused on the query's intent?

---

### **Calculation Approach**:

1. **Question Generation**: 
   - Use the LLM to generate artificial questions based on the response
   - Default is to create 3 variant questions
   - These questions should capture the essence of the response

2. **Semantic Similarity**:
   - Compute cosine similarity between:
     - Original user input embedding
     - Embeddings of generated questions
   - Measures how closely the questions match the original query

3. **Scoring**:
   - Average the cosine similarity scores
   - Higher scores indicate better relevance
   - Scores typically range between 0 and 1

---

### **Formula**:

**Response Relevancy** $\rightarrow$ $\frac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(E_{g_{i}}, E_{o})$ 

Where:
- $E_{g_{i}}$: Embedding of the i-th generated question
- $E_{o}$: Embedding of the original user input
- N: Number of generated questions (default: 3)

---

### **Evaluation Expectations**:
- **High Score (Close to 1)**: 
  - Answer directly addresses the query
  - Comprehensive and focused response
  - Minimal irrelevant information

- **Low Score (Close to 0)**: 
  - Response is off-topic
  - Incomplete or evasive answer
  - Includes excessive unrelated details

---

### **Important Limitations**:
- **No Factuality Check**: 
  - Measures relevance, not accuracy
  - Does not verify the truthfulness of the response
- **Embedding-Based**: 
  - Relies on semantic similarity
  - May not catch nuanced relevance

---

### **Example Scenario**:
- **User Query**: "When was the first Super Bowl?"
- **Good Response**: "The first Super Bowl was held on Jan 15, 1967, between the Green Bay Packers and Kansas City Chiefs."
- **Poor Response**: "Football is a popular sport in the United States with many interesting historical moments."

### **Key Insights**:
- Helps evaluate response quality beyond simple keyword matching
- Provides a quantitative measure of semantic alignment
- Supports improving AI system's query understanding
- Identifies potential issues with off-topic or unfocused responses

In [ ]:
from ragas.metrics import ResponseRelevancy

response_relevancy = ResponseRelevancy()

### **Faithfulness**

### TL;DR:
> **How factually consistent is the response with the retrieved context?**

---

### **Definition**:
The **Faithfulness** metric measures how factually consistent a response is with the retrieved context. It ranges from 0 to 1, with higher scores indicating better consistency.

A response is considered faithful if all its claims can be supported by the retrieved context.

---

### **Calculation Approach**:

1. **Identify Claims**: 
   - Break down the response into individual statements
   - Examine each claim systematically

2. **Context Verification**:
   - Check each claim to see if it can be inferred from the retrieved context
   - Determine the support level of each statement

3. **Scoring**:
   - Compute the faithfulness score using the formula:
     
     $\text{Faithfulness Score} = \frac{\text{Number of claims supported by the retrieved context}}{\text{Total number of claims in the response}}$

---

### **Example**:

**Question**: Where and when was Einstein born?

**Context**: Albert Einstein (born 14 March 1879) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time.

**High Faithfulness Answer**: Einstein was born in Germany on 14th March 1879.

**Low Faithfulness Answer**: Einstein was born in Germany on 20th March 1879.

#### Calculation Steps:
- **Step 1**: Break the generated answer into individual statements
- **Step 2**: Verify if each statement can be inferred from the given context
- **Step 3**: Apply the faithfulness formula

---

### **Evaluation Expectations**:
- **Perfect Score (1.0)**: 
  - All claims are directly supported by the context
  - No extraneous or unsupported information

- **Partial Score**: 
  - Some claims are supported
  - Partial consistency with the retrieved context

- **Low Score (Close to 0)**: 
  - Most claims cannot be verified
  - Significant deviation from the original context

---

### **Advanced Verification**:
**HEM-2.1-Open** can be used as a classifier model to:
- Detect hallucinations in LLM-generated text
- Cross-check claims with the given context
- Efficiently determine claim inferability

---

### **Key Insights**:
- Measures the factual consistency of AI-generated responses
- Helps identify potential hallucinations or fabrications
- Provides a quantitative assessment of contextual alignment
- Supports improving the reliability of AI-generated content

In [ ]:
from ragas.metrics import FaithfulnesswithHHEM

faithfulness = FaithfulnesswithHHEM(device="cuda:0")

You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
/home/p3tr0vv/Desktop/Evaluation-Approaches-for-Retrieval-Augmented-Generation-RAG-/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from ragas.metrics import FactualCorrectness

"""
Measures the factual consistency between the reference and the actual response by the LLM.

It uses true positives, false positives, false negatives.
TP = claim/s which is/are supported both by the reference and the response
FP = claim/s which is/are supported by the response, not by the reference
FN = claim/s which is/are supported by the reference, not response

Precision, Recall, and F1 modes

Precision = TP / (TP + FP) => everything which is in the response (even the redundant/missing data)
Recall = TP / (TP + FN) => all claims which are part and not part of the response
F1 = 2 * Precision * Recall / (Precision + Recall)

https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/factual_correctness/
"""

factual_correctness = FactualCorrectness(atomicity="high", coverage="high")

In [9]:
result = evaluate(
    dataset=eval_dataset,
    metrics=[context_precision, llm_context_recall, response_relevancy, faithfulness, factual_correctness],
    llm=llm,
    embeddings=embeddings,
    run_config=run_config
)

Evaluating: 100%|██████████| 260/260 [3:15:39<00:00, 45.15s/it]   


In [10]:
result_df = result.to_pandas()
result_df.to_csv('metrics_evaluation.csv', index=False)
result

{'llm_context_precision_with_reference': 0.7500, 'context_recall': 0.5730, 'answer_relevancy': 0.9423, 'faithfulness_with_hhem': 0.4200, 'factual_correctness(mode=f1)': 0.4952}

In [11]:
result.upload()

Evaluation results uploaded! View at https://app.ragas.io/dashboard/alignment/evaluation/4cd1785f-206c-474d-a92e-f3a90538aa51


'https://app.ragas.io/dashboard/alignment/evaluation/4cd1785f-206c-474d-a92e-f3a90538aa51'